# 🚀 Entraînement ICM sur Google Colab

Ce notebook entraîne un agent DQN avec **Curiosité (ICM)** pour la navigation robotique.

## 🎯 Configuration:
- GPU: T4/V100 (gratuit)
- Épisodes: 1500
- Environnement: NavigationEnv (800x600)
- ICM: Feature + Inverse + Forward Models

---

## 📦 1. Installation des Dépendances

⏱️ ~2 minutes

In [ ]:
!pip install -q torch torchvision gymnasium pygame numpy matplotlib seaborn pyyaml tqdm scipy pandas tensorboard

print("✅ Installation terminée!")

In [ ]:
import torch
print(f"🔥 CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   ⚠️  CPU mode")

## 📁 2. Cloner le Projet depuis GitHub

In [ ]:
!git clone https://github.com/mohhajji-1111/projet_RL.git
%cd projet_RL
!ls -la

## 🔧 3. Configuration Python Path

⚠️ **IMPORTANT**: Cette cellule configure le path pour que les imports fonctionnent

In [ ]:
import sys
import os
from pathlib import Path

# Configurer le Python path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")
print(f"✅ Python path configuré")

# Vérifier la structure
print("\n📁 Vérification des fichiers:")
if os.path.exists('src/agents/curiosity_agent.py'):
    print("   ✅ curiosity_agent.py trouvé")
if os.path.exists('configs/curiosity_config.yaml'):
    print("   ✅ curiosity_config.yaml trouvé")
if os.path.exists('src/environment/navigation_env.py'):
    print("   ✅ navigation_env.py trouvé")

print("\n🎯 Prêt pour l'entraînement!")

## 🎮 4. Test de l'Environnement

In [ ]:
from src.environment.navigation_env import NavigationEnv
import numpy as np

# Créer environnement
env = NavigationEnv(width=800, height=600, render_mode=None)

print("✅ Environnement créé!")
print(f"   Observation space: {env.observation_space.shape}")
print(f"   Action space: {env.action_space.n} actions")

# Test rapide
state, _ = env.reset()
print(f"\n📊 Test rapide (5 steps):")
for i in range(5):
    action = env.action_space.sample()
    state, reward, done, truncated, info = env.step(action)
    print(f"   Step {i+1}: action={action}, reward={reward:.2f}")

print("\n✅ Environnement OK!")

## 🧠 5. Créer l'Agent avec Curiosité

## ⚠️ VÉRIFICATION AVANT IMPORT

**IMPORTANT:** Si tu obtiens une erreur `ModuleNotFoundError`, vérifie que:
1. Tu as exécuté la **Cellule 5** (Configuration Python Path)
2. Tu es dans le bon répertoire (`projet_RL/`)
3. Le dossier `src/` existe

Exécute la cellule ci-dessous pour vérifier:

In [ ]:
import os
import sys

print("🔍 Vérification de l'environnement...\n")

# 1. Répertoire actuel
current_dir = os.getcwd()
print(f"📁 Répertoire: {current_dir}")

# 2. Vérifier la structure
checks = {
    'src/': os.path.exists('src'),
    'src/agents/': os.path.exists('src/agents'),
    'src/agents/curiosity_agent.py': os.path.exists('src/agents/curiosity_agent.py'),
    'configs/curiosity_config.yaml': os.path.exists('configs/curiosity_config.yaml'),
}

print("\n📋 Fichiers requis:")
for path, exists in checks.items():
    status = "✅" if exists else "❌"
    print(f"   {status} {path}")

# 3. Vérifier Python path
print(f"\n🐍 Python path:")
project_in_path = any('projet_RL' in p or current_dir in p for p in sys.path)
print(f"   {'✅' if project_in_path else '❌'} Project dans sys.path")
print(f"   sys.path[0]: {sys.path[0]}")

# 4. Verdict
if all(checks.values()) and project_in_path:
    print("\n✅ TOUT EST OK! Tu peux importer CuriosityAgent")
else:
    print("\n❌ PROBLÈME DÉTECTÉ!")
    if not all(checks.values()):
        print("   → Exécute la Cellule 4 (git clone)")
    if not project_in_path:
        print("   → Exécute la Cellule 5 (Configuration Python Path)")
    print("\n💡 Puis réexécute cette cellule de vérification")

In [ ]:
from src.agents.curiosity_agent import CuriosityAgent
import yaml

# Charger config
with open('configs/curiosity_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Créer agent
agent = CuriosityAgent(
    state_dim=env.observation_space.shape[0],
    action_dim=env.action_space.n,
    config=config['agent'],
    device=device
)

print("✅ CuriosityAgent créé!")
print(f"   Device: {device}")
print(f"   State dim: {env.observation_space.shape[0]}")
print(f"   Action dim: {env.action_space.n}")
print(f"   Feature dim: {config['agent']['feature_dim']}")
print(f"   Curiosity beta: {config['agent']['curiosity_beta']}")

print("\n📊 Paramètres des réseaux:")
print(f"   Feature Network: {sum(p.numel() for p in agent.feature_net.parameters()):,} params")
print(f"   Inverse Model: {sum(p.numel() for p in agent.inverse_model.parameters()):,} params")
print(f"   Forward Model: {sum(p.numel() for p in agent.forward_model.parameters()):,} params")
print(f"   Q-Network: {sum(p.numel() for p in agent.q_network.parameters()):,} params")

## 🚀 6. ENTRAÎNEMENT

🎯 Lance l'entraînement avec suivi en temps réel

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import deque

# Paramètres
num_episodes = 1500
eval_interval = 50
save_interval = 100

# Créer dossiers
os.makedirs('results/models/curiosity', exist_ok=True)
os.makedirs('results/plots', exist_ok=True)

# Métriques
episode_rewards = []
episode_lengths = []
intrinsic_rewards = []
forward_losses = []
inverse_losses = []
success_rate = deque(maxlen=100)
avg_rewards = deque(maxlen=100)
best_reward = -float('inf')

print("🚀 Début de l'entraînement...\n")

pbar = tqdm(range(num_episodes), desc="Training")

for episode in pbar:
    state, _ = env.reset()
    episode_reward = 0
    episode_intrinsic = 0
    episode_length = 0
    done = False
    truncated = False
    
    while not (done or truncated):
        # Action
        action = agent.select_action(state)
        
        # Step
        next_state, reward, done, truncated, info = env.step(action)
        
        # Récompense intrinsèque
        intrinsic_reward = agent.compute_intrinsic_reward(
            torch.FloatTensor(state).unsqueeze(0).to(device),
            torch.LongTensor([action]).to(device),
            torch.FloatTensor(next_state).unsqueeze(0).to(device)
        )
        
        # Stocker
        agent.store_transition(state, action, reward, next_state, done or truncated)
        
        # Entraîner
        if len(agent.replay_buffer) > agent.batch_size:
            loss = agent.train_step()
        
        episode_reward += reward
        episode_intrinsic += intrinsic_reward
        episode_length += 1
        state = next_state
    
    # Métriques
    episode_rewards.append(episode_reward)
    episode_lengths.append(episode_length)
    intrinsic_rewards.append(episode_intrinsic)
    avg_rewards.append(episode_reward)
    success_rate.append(1.0 if info.get('success', False) else 0.0)
    
    icm_stats = agent.get_icm_stats()
    forward_losses.append(icm_stats['forward_loss'])
    inverse_losses.append(icm_stats['inverse_loss'])
    
    # Progress
    pbar.set_postfix({
        'Reward': f"{episode_reward:.2f}",
        'Avg': f"{np.mean(avg_rewards):.2f}",
        'Success': f"{np.mean(success_rate):.1%}",
        'ε': f"{agent.epsilon:.3f}"
    })
    
    # Sauvegarder meilleur
    if episode_reward > best_reward:
        best_reward = episode_reward
        agent.save_checkpoint('results/models/curiosity/best.pth')
    
    # Checkpoints
    if (episode + 1) % save_interval == 0:
        agent.save_checkpoint(f'results/models/curiosity/checkpoint_{episode+1}.pth')
        print(f"\n💾 Checkpoint: episode {episode+1}")

print("\n" + "="*60)
print("✅ ENTRAÎNEMENT TERMINÉ!")
print("="*60)
print(f"Meilleure récompense: {best_reward:.2f}")
print(f"Taux de succès final: {np.mean(success_rate):.2%}")
print(f"Récompense moyenne (100 derniers): {np.mean(avg_rewards):.2f}")

## 📊 7. Visualisations

In [ ]:
import seaborn as sns
from scipy.ndimage import gaussian_filter1d

sns.set_style('whitegrid')
fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# 1. Récompenses
ax = axes[0, 0]
ax.plot(episode_rewards, alpha=0.3, color='blue')
smoothed = gaussian_filter1d(episode_rewards, sigma=20)
ax.plot(smoothed, color='red', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Reward')
ax.set_title('📈 Récompenses d\'Épisode')
ax.grid(alpha=0.3)

# 2. Moyenne mobile
ax = axes[0, 1]
window = 100
moving_avg = [np.mean(episode_rewards[max(0, i-window):i+1]) for i in range(len(episode_rewards))]
ax.plot(moving_avg, color='green', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Avg Reward')
ax.set_title(f'📊 Moyenne Mobile (n={window})')
ax.grid(alpha=0.3)

# 3. Curiosité
ax = axes[1, 0]
smoothed_int = gaussian_filter1d(intrinsic_rewards, sigma=20)
ax.plot(smoothed_int, color='orange', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Intrinsic Reward')
ax.set_title('🔍 Récompenses Intrinsèques')
ax.grid(alpha=0.3)

# 4. ICM Losses
ax = axes[1, 1]
ax.plot(gaussian_filter1d(forward_losses, sigma=20), label='Forward', color='purple', linewidth=2)
ax.plot(gaussian_filter1d(inverse_losses, sigma=20), label='Inverse', color='brown', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Loss')
ax.set_title('🧠 ICM Losses')
ax.legend()
ax.grid(alpha=0.3)

# 5. Longueur
ax = axes[2, 0]
smoothed_len = gaussian_filter1d(episode_lengths, sigma=20)
ax.plot(smoothed_len, color='darkblue', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Steps')
ax.set_title('⏱️ Longueur des Épisodes')
ax.grid(alpha=0.3)

# 6. Distribution
ax = axes[2, 1]
ax.hist(episode_rewards, bins=50, alpha=0.7, color='green', edgecolor='black')
ax.axvline(np.mean(episode_rewards), color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Reward')
ax.set_ylabel('Count')
ax.set_title('📊 Distribution des Récompenses')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/training_summary.png', dpi=150)
plt.show()

print("✅ Graphiques sauvegardés!")

## 📈 8. Statistiques Finales

In [ ]:
print("="*60)
print("📊 STATISTIQUES FINALES")
print("="*60)

print(f"\n🎯 Récompenses:")
print(f"   Moyenne: {np.mean(episode_rewards):.2f} ± {np.std(episode_rewards):.2f}")
print(f"   Médiane: {np.median(episode_rewards):.2f}")
print(f"   Min/Max: {np.min(episode_rewards):.2f} / {np.max(episode_rewards):.2f}")
print(f"   Meilleur: {best_reward:.2f}")

last_100 = episode_rewards[-100:]
print(f"\n📊 Performance (100 derniers):")
print(f"   Moyenne: {np.mean(last_100):.2f} ± {np.std(last_100):.2f}")
print(f"   Succès: {np.mean(list(success_rate)):.2%}")

print(f"\n⏱️ Épisodes:")
print(f"   Longueur moyenne: {np.mean(episode_lengths):.1f} steps")

print(f"\n🔍 ICM:")
print(f"   Récompense intrinsèque moy: {np.mean(intrinsic_rewards):.4f}")
print(f"   Forward loss finale: {forward_losses[-1]:.4f}")
print(f"   Inverse loss finale: {inverse_losses[-1]:.4f}")

print(f"\n🧠 Agent:")
print(f"   Epsilon final: {agent.epsilon:.4f}")
print(f"   Buffer size: {len(agent.replay_buffer)}")
print("\n" + "="*60)

## 💾 9. Télécharger les Résultats

In [ ]:
# Compresser
!zip -r results.zip results/

# Télécharger
from google.colab import files
files.download('results.zip')

print("✅ results.zip téléchargé!")
print("\nContenu:")
print("  📁 models/ - Modèles entraînés (.pth)")
print("  📁 plots/ - Graphiques (.png)")

## 🎉 Entraînement Terminé!

### Fichiers générés:
- ✅ `results/models/curiosity/best.pth` - Meilleur modèle
- ✅ `results/plots/training_summary.png` - Visualisations

### Prochaines étapes:
1. Télécharge `results.zip`
2. Teste le modèle localement avec l'interface GUI
3. Compare avec un agent DQN classique

**Bon courage! 🚀**